# ONNX and FFT

ONNX does not fully support complex yet. It does not have any FFT operators either. What if we need them anyway?

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [2]:
%load_ext mlprodict

In [3]:
import numpy
numpy.__version__

'1.22.1'

## Python implementation of RFFT

We try to replicate [numpy.rfft](https://numpy.org/doc/stable/reference/generated/numpy.fft.rfft.html).

In [4]:
import numpy


def almost_equal(a, b, error=1e-5):
    """
    The function compares two matrices, one may be complex. In that case,
    this matrix is changed into a new matrix with a new first dimension,
    [0,::] means real part, [1,::] means imaginary part.
    """
    if a.dtype in (numpy.complex64, numpy.complex128):
        dtype = numpy.float64 if a.dtype == numpy.complex128 else numpy.float32
        new_a = numpy.empty((2,) + a.shape).astype(dtype)
        new_a[0] = numpy.real(a)
        new_a[1] = numpy.imag(a)
        return almost_equal(new_a, b, error)
    if b.dtype in (numpy.complex64, numpy.complex128):
        return almost_equal(b, a, error)
    if a.shape != b.shape:
        raise AssertionError("Shape mismatch %r != %r." % (a.shape, b.shape))
    diff = numpy.abs(a.ravel() - b.ravel()).max()
    if diff > error:
        raise AssertionError("Mismatch max diff=%r > %r." % (diff, error))


def dft_real_cst(N, fft_length):
    n = numpy.arange(N)
    k = n.reshape((N, 1)).astype(numpy.float64)
    M = numpy.exp(-2j * numpy.pi * k * n / fft_length)
    both = numpy.empty((2,) + M.shape)
    both[0, :, :] = numpy.real(M)
    both[1, :, :] = numpy.imag(M)
    return both


def dft_real(x, fft_length=None, transpose=True):
    if len(x.shape) == 1:
        x = x.reshape((1, -1))
        N = 1
    else:
        N = x.shape[0]        
    C = x.shape[-1] if transpose else x.shape[-2]
    if fft_length is None:
        fft_length = x.shape[-1]
    size = fft_length // 2 + 1

    cst = dft_real_cst(C, fft_length)
    if transpose:
        x = numpy.transpose(x, (1, 0))
        a = cst[:, :, :fft_length]
        b = x[:fft_length]
        res = numpy.matmul(a, b)
        res = res[:, :size, :]
        return numpy.transpose(res, (0, 2, 1))
    else:
        a = cst[:, :, :fft_length]
        b = x[:fft_length]
        return numpy.matmul(a, b)


rnd = numpy.random.randn(5, 7).astype(numpy.float32)
fft_np = numpy.fft.rfft(rnd)
fft_cus = dft_real(rnd)
fft_np

array([[-0.75051495+0.j        ,  1.33475465+3.13179737j,
        -1.87604383+0.49239622j,  0.73039102+1.11186655j],
       [-0.76200065+0.j        ,  0.03706497+0.76383572j,
         2.22939392-1.13904508j, -3.81884915+0.45814935j],
       [ 1.2141826 +0.j        ,  0.50065353-2.12543076j,
        -3.10769194+0.43760207j, -0.91013869+0.88561919j],
       [-2.93666464+0.j        ,  1.07270369-2.56275325j,
        -1.42040003+0.10802866j, -1.65220639-2.11547056j],
       [ 2.98258114+0.j        , -1.71955059-1.29732326j,
         1.22977031+0.34286838j, -0.76048649+1.50560422j]])

Function `almost_equal` verifies both functions return the same results.

In [5]:
almost_equal(fft_np, fft_cus)

Let's do the same with `fft_length < shape[1]`.

In [6]:
fft_np3 = numpy.fft.rfft(rnd, n=3)
fft_cus3 = dft_real(rnd, fft_length=3)
fft_np3

array([[-1.10793678+0.j        ,  0.4741874 +0.66750763j],
       [-1.34050566+0.j        , -0.15834314-2.11824309j],
       [ 1.72974443+0.j        , -2.11205184+0.59408175j],
       [-0.06082912+0.j        , -1.45597189-1.08558287j],
       [ 1.1374151 +0.j        , -0.46541163+0.50106358j]])

In [7]:
almost_equal(fft_np3, fft_cus3)

## RFFT in ONNX

Let's assume first the number of column of the input matrix is fixed. The result of function `dft_real_cst` can be considered as constant.

In [8]:
from typing import Any
import mlprodict.npy.numpy_onnx_impl as npnx
from mlprodict.npy import onnxnumpy_np
from mlprodict.npy.onnx_numpy_annotation import NDArrayType
# from mlprodict.onnxrt import OnnxInference

@onnxnumpy_np(signature=NDArrayType(("T:all", ), dtypes_out=('T',)))
def onnx_rfft(x, fft_length=None):
    if fft_length is None:
        raise RuntimeError("fft_length must be specified.")
    
    size = fft_length // 2 + 1
    cst = dft_real_cst(fft_length, fft_length).astype(numpy.float32)
    xt = npnx.transpose(x, (1, 0))
    res = npnx.matmul(cst[:, :, :fft_length], xt[:fft_length])[:, :size, :]
    return npnx.transpose(res, (0, 2, 1))

fft_onx = onnx_rfft(rnd, fft_length=rnd.shape[1])
fft_onx

No CUDA runtime is found, using CUDA_HOME='C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.5'


array([[[-0.75051486,  1.3347546 , -1.8760438 ,  0.730391  ],
        [-0.7620005 ,  0.03706497,  2.2293937 , -3.8188488 ],
        [ 1.2141826 ,  0.50065356, -3.107692  , -0.9101387 ],
        [-2.9366646 ,  1.0727037 , -1.4204    , -1.6522063 ],
        [ 2.9825811 , -1.7195507 ,  1.2297703 , -0.7604865 ]],

       [[ 0.        ,  3.1317973 ,  0.49239618,  1.1118665 ],
        [ 0.        ,  0.7638357 , -1.1390451 ,  0.4581493 ],
        [ 0.        , -2.1254308 ,  0.437602  ,  0.88561916],
        [ 0.        , -2.5627534 ,  0.10802869, -2.1154706 ],
        [ 0.        , -1.2973232 ,  0.34286833,  1.5056041 ]]],
      dtype=float32)

In [9]:
almost_equal(fft_cus, fft_onx)

The corresponding ONNX graph is the following:

In [10]:
%onnxview onnx_rfft.to_onnx()

In [11]:
fft_onx3 = onnx_rfft(rnd, fft_length=3)
almost_equal(fft_cus3, fft_onx3)

## FFT 2D

Below the code for complex features.

In [12]:
def _DFT_cst(N, fft_length, trunc=True):
    n = numpy.arange(N)
    k = n.reshape((N, 1)).astype(numpy.float64)
    M = numpy.exp(-2j * numpy.pi * k * n / fft_length)
    return M[:fft_length // 2 + 1] if trunc else M

def DFT(x, fft_length=None, axis=1):
    if axis == 1:
        x = x.T
    if fft_length is None:
        fft_length = x.shape[0]
    cst = _DFT_cst(x.shape[0], fft_length, trunc=axis==1)
    if axis == 1:
        return numpy.matmul(cst, x).T
    return numpy.matmul(cst, x)

def fft2d_(mat, fft_length):
    mat = mat[:fft_length[0], :fft_length[1]]
    res = mat.copy()
    res = DFT(res, fft_length[1], axis=1)
    res = DFT(res, fft_length[0], axis=0)
    return res[:fft_length[0], :fft_length[1]//2 + 1]


rnd = numpy.random.randn(5, 7).astype(numpy.float32)
fft2d_np_ = fft2d_(rnd, rnd.shape)
fft2d_np = numpy.fft.rfft2(rnd)
fft2d_np_

array([[10.14779193+0.j        , -1.55134743-1.67220211j,
         3.23984394-7.15630184j, -8.55688343-3.86255514j],
       [-5.47836537-7.29111075j, -1.44693655-4.16582164j,
        -0.15482528-2.91009629j, -5.74598895+0.4286478j ],
       [ 0.37482297-4.60517198j, -3.39913789+3.28850098j,
        -4.56144176+9.89934275j,  5.52501083-1.01792048j],
       [ 0.37482297+4.60517198j, -4.16542648-0.19354436j,
         1.59761181-7.05603298j,  0.02120182+6.56731064j],
       [-5.47836537+7.29111075j,  2.29746565+7.31672206j,
         1.93122673+1.08542613j, -2.22944407+2.13972051j]])

In [13]:
almost_equal(fft2d_np_, fft2d_np)

It implies the computation of two FFT 1D along both axes. However, as ONNX does not support complex, it needs to be rewritten with only real numbers. The algorithm can be summarized into this formula $FFT(FFT(x, axis=1), axis=0)$. If *x* is real, $FFT(x, .)$ is complex. We still assume *x* is real, it then becomes (FFT is a linear operator, so $FFT(ix)=i FFT(x)$):

* $y = FFT(x, axis=1)$
* $z_r = FFT(Real(y), axis=0)$, $z_i = FFT(Imag(y), axis=0)$
* $z = z_r + i z_i$

*z* is the desired output. The following implementation is probably not the most efficient one. It avoids inplace computation as ONNX does like that.

In [14]:
def fft2d(mat, fft_length):
    mat = mat[:fft_length[0], :fft_length[1]]
    res = mat.copy()
    
    # first FFT
    res = dft_real(res, fft_length=fft_length[1], transpose=True)
    
    # second FFT decomposed on FFT on real part and imaginary part
    res2_real = dft_real(res[0], fft_length=fft_length[0], transpose=False)
    res2_imag = dft_real(res[1], fft_length=fft_length[0], transpose=False)    
    res2_imag2 = numpy.vstack([-res2_imag[1:2], res2_imag[:1]])
    res = res2_real + res2_imag2
    size = fft_length[1]//2 + 1
    return res[:, :fft_length[0], :size]


fft2d_np = numpy.fft.rfft2(rnd)
fft2d_cus = fft2d(rnd, rnd.shape)
almost_equal(fft2d_np, fft2d_cus)

In [15]:
fft2d_np

array([[10.14779193+0.j        , -1.55134743-1.67220211j,
         3.23984394-7.15630184j, -8.55688343-3.86255514j],
       [-5.47836537-7.29111075j, -1.44693655-4.16582164j,
        -0.15482528-2.91009629j, -5.74598895+0.4286478j ],
       [ 0.37482297-4.60517198j, -3.39913789+3.28850098j,
        -4.56144176+9.89934275j,  5.52501083-1.01792048j],
       [ 0.37482297+4.60517198j, -4.16542648-0.19354436j,
         1.59761181-7.05603298j,  0.02120182+6.56731064j],
       [-5.47836537+7.29111075j,  2.29746565+7.31672206j,
         1.93122673+1.08542613j, -2.22944407+2.13972051j]])

In [16]:
fft2d_cus

array([[[10.14779193, -1.55134743,  3.23984394, -8.55688343],
        [-5.47836537, -1.44693655, -0.15482528, -5.74598895],
        [ 0.37482297, -3.39913789, -4.56144176,  5.52501083],
        [ 0.37482297, -4.16542648,  1.59761181,  0.02120182],
        [-5.47836537,  2.29746565,  1.93122673, -2.22944407]],

       [[ 0.        , -1.67220211, -7.15630184, -3.86255514],
        [-7.29111075, -4.16582164, -2.91009629,  0.4286478 ],
        [-4.60517198,  3.28850098,  9.89934275, -1.01792048],
        [ 4.60517198, -0.19354436, -7.05603298,  6.56731064],
        [ 7.29111075,  7.31672206,  1.08542613,  2.13972051]]])

And with a different `fft_length`.

In [17]:
fft2d_np = numpy.fft.rfft2(rnd, (4, 6))
fft2d_cus = fft2d(rnd, (4, 6))
almost_equal(fft2d_np[:4, :], fft2d_cus)

## FFT 2D in ONNX

We use again the numpy API for ONNX.

In [18]:
def onnx_rfft_1d(x, fft_length=None, transpose=True):
    if fft_length is None:
        raise RuntimeError("fft_length must be specified.")
    
    size = fft_length // 2 + 1
    cst = dft_real_cst(fft_length, fft_length).astype(numpy.float32)
    if transpose:
        xt = npnx.transpose(x, (1, 0))
        res = npnx.matmul(cst[:, :, :fft_length], xt[:fft_length])[:, :size, :]
        return npnx.transpose(res, (0, 2, 1))
    else:
        return npnx.matmul(cst[:, :, :fft_length], x[:fft_length])


@onnxnumpy_np(signature=NDArrayType(("T:all", ), dtypes_out=('T',)))
def onnx_rfft_2d(x, fft_length=None):
    mat = x[:fft_length[0], :fft_length[1]]
    
    # first FFT
    res = onnx_rfft_1d(mat, fft_length=fft_length[1], transpose=True)
    
    # second FFT decomposed on FFT on real part and imaginary part
    res2_real = onnx_rfft_1d(res[0], fft_length=fft_length[0], transpose=False)
    res2_imag = onnx_rfft_1d(res[1], fft_length=fft_length[0], transpose=False)    
    res2_imag2 = npnx.vstack(-res2_imag[1:2], res2_imag[:1])
    res = res2_real + res2_imag2
    size = fft_length[1]//2 + 1
    return res[:, :fft_length[0], :size]


fft2d_cus = fft2d(rnd, rnd.shape)
fft2d_onx = onnx_rfft_2d(rnd, fft_length=rnd.shape)
almost_equal(fft2d_cus, fft2d_onx)

The corresponding ONNX graph.

In [19]:
%onnxview onnx_rfft_2d.to_onnx()

In [20]:
with open("fft2d.onnx", "wb") as f:
    f.write(onnx_rfft_2d.to_onnx().SerializeToString())

With a different `fft_length`.

In [21]:
fft2d_cus = fft2d(rnd, (4, 5))
fft2d_onx = onnx_rfft_2d(rnd, fft_length=(4, 5))
almost_equal(fft2d_cus, fft2d_onx)

This implementation of FFT in ONNX assumes shapes and fft lengths are constant. Otherwise, the matrix returned by function `dft_real_cst` must be converted as well. That's left as an exercise.

## FFT2D with shape (3,1,4)

Previous implementation expects the input matrix to have two dimensions. It fails with 3.

In [22]:
shape = (3, 1, 4)
fft_length = (1, 4)
rnd = numpy.random.randn(*list(shape)).astype(numpy.float32)
fft2d_numpy = numpy.fft.fft2(rnd, fft_length)
fft2d_numpy.shape

(3, 1, 4)

In [23]:
fft2d_numpy

array([[[ 1.34545542+0.j        , -0.35498468-0.77279791j,
         -1.72351556+0.j        , -0.35498468+0.77279791j]],

       [[-1.13275981+0.j        , -2.43033203+0.77264115j,
         -2.97744519+0.j        , -2.43033203-0.77264115j]],

       [[-0.06647013+0.j        ,  0.20555305+1.91183175j,
          2.91867135+0.j        ,  0.20555305-1.91183175j]]])

In [24]:
try:
    fft2d_cus = fft2d(rnd, fft_length)
except Exception as e:
    print(e)
# fft2d_onx = onnx_rfft_2d(rnd, fft_length=fft_length)

axes don't match array


### numpy version

Let's do it again with numpy first. [fft2](https://numpy.org/doc/stable/reference/generated/numpy.fft.fft2.html) performs `fft2` on the last two axis as many times as the first axis. The goal is still to have an implementation which works for any dimension.

In [25]:
conc = []
for i in range(rnd.shape[0]):
    f2 = fft2d(rnd[i], fft_length)
    conc.append(numpy.expand_dims(f2, 0))
res = numpy.vstack(conc).transpose(1, 0, 2, 3)
almost_equal(fft2d_numpy[:, :, :3], res)

It works. And now a more efficient implementation. It is better to read [matmul](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html) description before. To summarize, a third axis is equivalent to many matrix multiplications over the last two axes, as many as the dimension of the first axis: ``matmul(A[I,J,K], B[I,K,L]) --> C[I,J,L]``. Broadcasting also works... ``matmul(A[1,J,K], B[I,K,L]) --> C[I,J,L]``.

In [26]:
def dft_real_d3(x, fft_length=None, transpose=True):
    if len(x.shape) != 3:
        raise RuntimeError("Not implemented for shape=%r." % x.shape)
    N = x.shape[1]
    C = x.shape[-1] if transpose else x.shape[-2]
    if fft_length is None:
        fft_length = x.shape[-1]
    size = fft_length // 2 + 1

    cst = dft_real_cst(C, fft_length)
    if transpose:
        x = numpy.transpose(x, (0, 2, 1))
        a = cst[:, :, :fft_length]
        b = x[:, :fft_length, :]
        a = numpy.expand_dims(a, 0)
        b = numpy.expand_dims(b, 1)
        res = numpy.matmul(a, b)
        res = res[:, :, :size, :]
        return numpy.transpose(res, (1, 0, 3, 2))
    else:
        a = cst[:, :, :fft_length]
        b = x[:, :fft_length, :]
        a = numpy.expand_dims(a, 0)
        b = numpy.expand_dims(b, 1)
        res = numpy.matmul(a, b)
        return numpy.transpose(res, (1, 0, 2, 3))


def fft2d_d3(mat, fft_length):
    mat = mat[:, :fft_length[-2], :fft_length[-1]]
    res = mat.copy()
    
    # first FFT
    res = dft_real_d3(res, fft_length=fft_length[-1], transpose=True)
    
    # second FFT decomposed on FFT on real part and imaginary part
    res2_real = dft_real_d3(res[0], fft_length=fft_length[-2], transpose=False)
    res2_imag = dft_real_d3(res[1], fft_length=fft_length[-2], transpose=False)
    res2_imag2 = numpy.vstack([-res2_imag[1:2], res2_imag[:1]])
    res = res2_real + res2_imag2
    size = fft_length[-1]//2 + 1
    return res[:, :, :fft_length[-2], :size]


def fft2d_any(mat, fft_length):
    new_shape = (-1, ) + mat.shape[-2:]
    mat2 = mat.reshape(new_shape)
    f2 = fft2d_d3(mat2, fft_length)
    new_shape = (2, ) + mat.shape[:-2] + f2.shape[-2:]
    return f2.reshape(new_shape)


shape = (3, 1, 4)
fft_length = (1, 4)
rnd = numpy.random.randn(*list(shape)).astype(numpy.float32)
fft2d_numpy = numpy.fft.fft2(rnd, fft_length)
fft2d_cus = fft2d_any(rnd, fft_length)
almost_equal(fft2d_numpy[..., :3], fft2d_cus)

We check with more shapes to see if the implementation works for all of them.

In [27]:
for shape in [(3, 1, 4), (5, 7), (3, 5, 7), (7, 5)]:
    for fft_length in [shape[-2:], (1, shape[-1]),
                       (min(2, shape[-2]), shape[-1]),
                       (shape[-2], 2),
                       (min(3, shape[-2]), min(4, shape[-2]))]:
        x  = numpy.random.randn(*list(shape)).astype(numpy.float32)
        fnp = numpy.fft.fft2(x, fft_length)
        if len(fnp.shape) == 2:
            fn= numpy.expand_dims(fnp, 0)
        try:
            cus = fft2d_any(x, fft_length)
        except IndexError as e:
            print("ERR x.shape=%r length=%r error=%r" % (x.shape, fft_length, e))
            continue
        try:
            almost_equal(fnp[..., :cus.shape[-1]], cus)
        except (AssertionError, IndexError) as e:
            print("DIS x.shape=%r length=%r error=%r  output shape=%r or %r" % (
                x.shape, fft_length, e, fnp.shape, cus.shape))
            continue
        print("OK  x.shape=%r length=%r output shape=%r or %r" % (
            x.shape, fft_length, fnp.shape, cus.shape))

OK  x.shape=(3, 1, 4) length=(1, 4) output shape=(3, 1, 4) or (2, 3, 1, 3)
OK  x.shape=(3, 1, 4) length=(1, 4) output shape=(3, 1, 4) or (2, 3, 1, 3)
OK  x.shape=(3, 1, 4) length=(1, 4) output shape=(3, 1, 4) or (2, 3, 1, 3)
OK  x.shape=(3, 1, 4) length=(1, 2) output shape=(3, 1, 2) or (2, 3, 1, 2)
OK  x.shape=(3, 1, 4) length=(1, 1) output shape=(3, 1, 1) or (2, 3, 1, 1)
OK  x.shape=(5, 7) length=(5, 7) output shape=(5, 7) or (2, 5, 4)
OK  x.shape=(5, 7) length=(1, 7) output shape=(1, 7) or (2, 1, 4)
OK  x.shape=(5, 7) length=(2, 7) output shape=(2, 7) or (2, 2, 4)
OK  x.shape=(5, 7) length=(5, 2) output shape=(5, 2) or (2, 5, 2)
OK  x.shape=(5, 7) length=(3, 4) output shape=(3, 4) or (2, 3, 3)
OK  x.shape=(3, 5, 7) length=(5, 7) output shape=(3, 5, 7) or (2, 3, 5, 4)
OK  x.shape=(3, 5, 7) length=(1, 7) output shape=(3, 1, 7) or (2, 3, 1, 4)
OK  x.shape=(3, 5, 7) length=(2, 7) output shape=(3, 2, 7) or (2, 3, 2, 4)
OK  x.shape=(3, 5, 7) length=(5, 2) output shape=(3, 5, 2) or (2, 3, 5

### ONNX version

Let's look into the differences first.

In [28]:
%load_ext pyquickhelper

In [29]:
%%html
<style>
table td, table th, table tr {text-align:left !important; white-space: pre;}
</style>

In [30]:
import inspect
text1 = inspect.getsource(dft_real)
text2 = inspect.getsource(dft_real_d3)
%codediff text1 text2 --verbose 1 --two 1

100%|██████████| 24/24 [00:00<00:00, 292.18it/s]


0,0,"def dft_real(x, fft_length=None, transpose=True):","def dft_real_d3(x, fft_length=None, transpose=True):"
1,1,if len(x.shape) == 1:,if len(x.shape) != 3:
2,2,"x = x.reshape((1, -1))","raise RuntimeError(""Not implemented for shape=%r."" % x.shape)"
3,,N = 1,
4,,else:,
5,3,N = x.shape[0],N = x.shape[1]
6,4,C = x.shape[-1] if transpose else x.shape[-2],C = x.shape[-1] if transpose else x.shape[-2]
7,5,if fft_length is None:,if fft_length is None:
8,6,fft_length = x.shape[-1],fft_length = x.shape[-1]
9,7,size = fft_length // 2 + 1,size = fft_length // 2 + 1
10,8,,


In [31]:
text1 = inspect.getsource(fft2d)
text2 = inspect.getsource(fft2d_d3)
%codediff text1 text2 --verbose 1 --two 1

100%|██████████| 15/15 [00:00<00:00, 357.70it/s]


0,0,"def fft2d(mat, fft_length):","def fft2d_d3(mat, fft_length):"
1,1,"mat = mat[:fft_length[0], :fft_length[1]]","mat = mat[:, :fft_length[-2], :fft_length[-1]]"
2,2,res = mat.copy(),res = mat.copy()
3,3,,
4,4,# first FFT,# first FFT
5,5,"res = dft_real(res, fft_length=fft_length[1], transpose=True)","res = dft_real_d3(res, fft_length=fft_length[-1], transpose=True)"
6,6,,
7,7,# second FFT decomposed on FFT on real part and imaginary part,# second FFT decomposed on FFT on real part and imaginary part
8,8,"res2_real = dft_real(res[0], fft_length=fft_length[0], transpose=False)","res2_real = dft_real_d3(res[0], fft_length=fft_length[-2], transpose=False)"
9,9,"res2_imag = dft_real(res[1], fft_length=fft_length[0], transpose=False)","res2_imag = dft_real_d3(res[1], fft_length=fft_length[-2], transpose=False)"
10,10,"res2_imag2 = numpy.vstack([-res2_imag[1:2], res2_imag[:1]])","res2_imag2 = numpy.vstack([-res2_imag[1:2], res2_imag[:1]])"


In [32]:
def onnx_rfft_3d_1d(x, fft_length=None, transpose=True):
    if fft_length is None:
        raise RuntimeError("fft_length must be specified.")
    
    size = fft_length // 2 + 1
    cst = dft_real_cst(fft_length, fft_length).astype(numpy.float32)
    if transpose:
        xt = npnx.transpose(x, (0, 2, 1))
        a = cst[:, :, :fft_length]
        b = xt[:, :fft_length, :]
        a = npnx.expand_dims(a, 0)
        b = npnx.expand_dims(b, 1)
        res = npnx.matmul(a, b)
        res2 = res[:, :size, :]
        return npnx.transpose(res2, (1, 0, 3, 2))
    else:
        a = cst[:, :, :fft_length]
        b = x[:, :fft_length, :]
        a = npnx.expand_dims(a, 0)
        b = npnx.expand_dims(b, 1)
        res = npnx.matmul(a, b)
        return npnx.transpose(res, (1, 0, 2, 3))      
    

def onnx_rfft_3d_2d(x, fft_length=None):
    mat = x[:, :fft_length[-2], :fft_length[-1]]
    
    # first FFT
    res = onnx_rfft_3d_1d(mat, fft_length=fft_length[-1], transpose=True)
    
    # second FFT decomposed on FFT on real part and imaginary part
    res2_real = onnx_rfft_3d_1d(res[0], fft_length=fft_length[0], transpose=False)
    res2_imag = onnx_rfft_3d_1d(res[1], fft_length=fft_length[0], transpose=False)    
    res2_imag2 = npnx.vstack(-res2_imag[1:2], res2_imag[:1])
    res = res2_real + res2_imag2
    size = fft_length[1]//2 + 1
    return res[:, :, :fft_length[-2], :size]


@onnxnumpy_np(signature=NDArrayType(("T:all", ), dtypes_out=('T',)))
def onnx_rfft_2d_any(x, fft_length=None):
    new_shape = npnx.concat(
        numpy.array([-1], dtype=numpy.int64), x.shape[-2:], axis=0)
    mat2 = x.reshape(new_shape)
    f2 = onnx_rfft_3d_2d(mat2, fft_length)
    new_shape = npnx.concat(
        numpy.array([2], dtype=numpy.int64), x.shape[:-2], f2.shape[-2:])
    return f2.reshape(new_shape)


shape = (3, 1, 4)
fft_length = (1, 4)
rnd = numpy.random.randn(*list(shape)).astype(numpy.float32)
fft2d_cus = fft2d_any(rnd, fft_length)
fft2d_onx = onnx_rfft_2d_any(rnd, fft_length=fft_length)
almost_equal(fft2d_cus, fft2d_onx)

Let's do the same comparison.

In [33]:
for shape in [(3, 1, 4), (5, 7), (3, 5, 7), (7, 5)]:
    for fft_length in [shape[-2:], (1, shape[-1]),
                       (min(2, shape[-2]), shape[-1]),
                       (shape[-2], 2),
                       (min(3, shape[-2]), min(4, shape[-2]))]:
        x  = numpy.random.randn(*list(shape)).astype(numpy.float32)
        if len(fnp.shape) == 2:
            fn= numpy.expand_dims(fnp, 0)
        try:
            cus = fft2d_any(x, fft_length)
        except IndexError as e:
            print("ERR x.shape=%r length=%r error=%r" % (x.shape, fft_length, e))
            continue
        try:
            onx = onnx_rfft_2d_any(x, fft_length=fft_length)
        except IndexError as e:
            print("ERR x.shape=%r length=%r error=%r" % (x.shape, fft_length, e))
            continue
        try:
            almost_equal(onx, cus)
        except (AssertionError, IndexError) as e:
            print("DIS x.shape=%r length=%r error=%r  output shape=%r or %r" % (
                x.shape, fft_length, e, fnp.shape, cus.shape))
            continue
        print("OK  x.shape=%r length=%r output shape=%r or %r" % (
            x.shape, fft_length, fnp.shape, cus.shape))

OK  x.shape=(3, 1, 4) length=(1, 4) output shape=(3, 4) or (2, 3, 1, 3)
OK  x.shape=(3, 1, 4) length=(1, 4) output shape=(3, 4) or (2, 3, 1, 3)
OK  x.shape=(3, 1, 4) length=(1, 4) output shape=(3, 4) or (2, 3, 1, 3)
OK  x.shape=(3, 1, 4) length=(1, 2) output shape=(3, 4) or (2, 3, 1, 2)
DIS x.shape=(3, 1, 4) length=(1, 1) error=AssertionError('Mismatch max diff=1.0 > 1e-05.')  output shape=(3, 4) or (2, 3, 1, 1)
OK  x.shape=(5, 7) length=(5, 7) output shape=(3, 4) or (2, 5, 4)
OK  x.shape=(5, 7) length=(1, 7) output shape=(3, 4) or (2, 1, 4)
OK  x.shape=(5, 7) length=(2, 7) output shape=(3, 4) or (2, 2, 4)
OK  x.shape=(5, 7) length=(5, 2) output shape=(3, 4) or (2, 5, 2)
OK  x.shape=(5, 7) length=(3, 4) output shape=(3, 4) or (2, 3, 3)
OK  x.shape=(3, 5, 7) length=(5, 7) output shape=(3, 4) or (2, 3, 5, 4)
OK  x.shape=(3, 5, 7) length=(1, 7) output shape=(3, 4) or (2, 3, 1, 4)
OK  x.shape=(3, 5, 7) length=(2, 7) output shape=(3, 4) or (2, 3, 2, 4)
OK  x.shape=(3, 5, 7) length=(5, 2) ou

There is one issue with ``fft_length=(1, 1)`` but that case is out of scope.

### ONNX graph

In [34]:
key = list(onnx_rfft_2d_any.signed_compiled)[0]
%onnxview onnx_rfft_2d_any.signed_compiled[key].compiled.onnx_

In [35]:
with open("fft2d_any.onnx", "wb") as f:
    key = list(onnx_rfft_2d_any.signed_compiled)[0]
    f.write(onnx_rfft_2d_any.signed_compiled[key].compiled.onnx_.SerializeToString())

Let's check the intermediate results.

In [36]:
key = list(onnx_rfft_2d_any.signed_compiled)[0]
key

FctVersion((numpy.float32,), ((1, 4),))

In [37]:
from mlprodict.onnxrt import OnnxInference

x = numpy.random.randn(3, 1, 4).astype(numpy.float32)
onx = onnx_rfft_2d_any.signed_compiled[key].compiled.onnx_
oinf = OnnxInference(onx)
oinf.run({'x': x}, verbose=1, fLOG=print)

+ki='Un_Unsqueezecst': (2, 1, 1) (dtype=float32 min=0.0 max=1.0)
+ki='Un_Unsqueezecst1': (1,) (dtype=int64 min=0 max=0)
+ki='Un_Unsqueezecst2': (2, 4, 4) (dtype=float32 min=-1.0 max=1.0)
+ki='Co_Concatcst': (1,) (dtype=int64 min=-1 max=-1)
+ki='Sl_Slicecst': (1,) (dtype=int64 min=-2 max=-2)
+ki='Sl_Slicecst2': (2,) (dtype=int64 min=0 max=0)
+ki='Sl_Slicecst3': (2,) (dtype=int64 min=1 max=4)
+ki='Sl_Slicecst4': (2,) (dtype=int64 min=1 max=2)
+ki='Sl_Slicecst6': (1,) (dtype=int64 min=4 max=4)
+ki='Sl_Slicecst7': (1,) (dtype=int64 min=1 max=1)
+ki='Sl_Slicecst9': (1,) (dtype=int64 min=3 max=3)
+ki='Ga_Gathercst1': () (dtype=int64 min=0 max=0)
+ki='Ga_Gathercst2': () (dtype=int64 min=1 max=1)
+ki='Sl_Slicecst18': (1,) (dtype=int64 min=2 max=2)
+ki='Sl_Slicecst24': (2,) (dtype=int64 min=1 max=3)
+ki='Sl_Slicecst25': (2,) (dtype=int64 min=2 max=3)
-- OnnxInference: run 38 nodes
Onnx-Unsqueeze(Un_Unsqueezecst, Un_Unsqueezecst1) -> Un_expanded0    (name='Un_Unsqueeze')
+kr='Un_expanded0': (1, 

{'y': array([[[[ 1.9479027e+00, -7.0837361e-01, -3.1971555e+00]],
 
         [[-2.4413235e+00, -1.2641068e+00, -3.5156422e+00]],
 
         [[ 1.9243780e-01,  6.0331464e-01,  2.6367991e+00]]],
 
 
        [[[ 0.0000000e+00,  7.5195622e-01, -7.1191992e-16]],
 
         [[ 0.0000000e+00,  1.5755669e+00, -5.3446789e-16]],
 
         [[ 0.0000000e+00,  5.3400773e-01,  3.3330694e-16]]]],
       dtype=float32)}